# 🏠 Análisis Exploratorio de Datos - Mercado Inmobiliario

**Objetivo:** Entender las características del dataset de propiedades y identificar patrones clave para el modelo predictivo.

**Fecha:** Junio 2025  
**Autor:** Tu Nombre

## 📋 Tabla de Contenido

1. [Carga de Datos](#1-carga-de-datos)
2. [Información General](#2-información-general)
3. [Análisis Univariado](#3-análisis-univariado)
4. [Análisis Bivariado](#4-análisis-bivariado)
5. [Detección de Outliers](#5-detección-de-outliers)
6. [Correlaciones](#6-correlaciones)
7. [Conclusiones](#7-conclusiones)

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("📚 Librerías importadas exitosamente")

## 1. Carga de Datos

In [ ]:
# Cargar dataset
df = pd.read_csv('../data/sample.csv')

print(f"📊 Dataset cargado exitosamente")
print(f"Forma del dataset: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"Período de datos: {df['fecha_publicacion'].min()} a {df['fecha_publicacion'].max()}")

## 2. Información General

In [ ]:
# Vista general del dataset
df.head()

In [ ]:
# Información detallada
print("🔍 INFORMACIÓN DEL DATASET")
print("=" * 40)
df.info()

In [ ]:
# Estadísticas descriptivas
print("📈 ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 40)
df.describe()

In [ ]:
# Valores faltantes
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Columna': missing_data.index,
    'Valores Faltantes': missing_data.values,
    'Porcentaje': missing_percent.values
})

print("❌ VALORES FALTANTES")
print("=" * 40)
print(missing_df[missing_df['Valores Faltantes'] > 0].sort_values('Porcentaje', ascending=False))

## 3. Análisis Univariado

### 3.1 Variable Objetivo: Precio

In [ ]:
# Distribución del precio
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma
axes[0].hist(df['precio'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('📊 Distribución de Precios', fontsize=14)
axes[0].set_xlabel('Precio (USD)')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(df['precio'])
axes[1].set_title('📦 Box Plot - Precios', fontsize=14)
axes[1].set_ylabel('Precio (USD)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas del precio
print(f"💰 Precio promedio: ${df['precio'].mean():,.0f} USD")
print(f"📊 Precio mediano: ${df['precio'].median():,.0f} USD")
print(f"📈 Precio máximo: ${df['precio'].max():,.0f} USD")
print(f"📉 Precio mínimo: ${df['precio'].min():,.0f} USD")

### 3.2 Variables Categóricas

In [ ]:
# Distribución por tipo de propiedad
plt.figure(figsize=(10, 6))
tipo_counts = df['tipo_propiedad'].value_counts()

plt.pie(tipo_counts.values, labels=tipo_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('🏢 Distribución por Tipo de Propiedad', fontsize=14)
plt.axis('equal')
plt.show()

print("🏠 TIPOS DE PROPIEDAD:")
for tipo, count in tipo_counts.items():
    print(f"  {tipo}: {count} propiedades ({count/len(df)*100:.1f}%)")

In [ ]:
# Top 10 barrios con más propiedades
plt.figure(figsize=(12, 6))
top_barrios = df['barrio'].value_counts().head(10)

sns.barplot(x=top_barrios.values, y=top_barrios.index, palette='viridis')
plt.title('🗺️ Top 10 Barrios con Más Propiedades', fontsize=14)
plt.xlabel('Cantidad de Propiedades')
plt.ylabel('Barrio')

# Agregar valores en las barras
for i, v in enumerate(top_barrios.values):
    plt.text(v + 1, i, str(v), va='center')

plt.tight_layout()
plt.show()

## 4. Análisis Bivariado

### 4.1 Precio vs Superficie

In [ ]:
# Scatter plot: Precio vs Superficie
plt.figure(figsize=(12, 8))

# Scatter plot con regresión
sns.scatterplot(data=df, x='superficie', y='precio', hue='tipo_propiedad', alpha=0.6)
sns.regplot(data=df, x='superficie', y='precio', scatter=False, color='red', line_kws={'linewidth': 2})

plt.title('🏠 Relación entre Superficie y Precio', fontsize=16)
plt.xlabel('Superficie (m²)')
plt.ylabel('Precio (USD)')
plt.legend(title='Tipo de Propiedad', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Correlación
correlacion = df['precio'].corr(df['superficie'])
print(f"🔗 Correlación Precio-Superficie: {correlacion:.3f}")

### 4.2 Precio por Barrio

In [ ]:
# Precio promedio por barrio (top 15)
precio_por_barrio = df.groupby('barrio')['precio'].agg(['mean', 'count']).reset_index()
precio_por_barrio = precio_por_barrio[precio_por_barrio['count'] >= 5]  # Solo barrios con 5+ propiedades
precio_por_barrio = precio_por_barrio.sort_values('mean', ascending=False).head(15)

plt.figure(figsize=(14, 8))
sns.barplot(data=precio_por_barrio, x='mean', y='barrio', palette='coolwarm')
plt.title('💰 Precio Promedio por Barrio (Top 15)', fontsize=16)
plt.xlabel('Precio Promedio (USD)')
plt.ylabel('Barrio')

# Agregar valores
for i, v in enumerate(precio_por_barrio['mean']):
    plt.text(v + 5000, i, f'${v:,.0f}', va='center')

plt.tight_layout()
plt.show()

## 5. Detección de Outliers

In [ ]:
# Detectar outliers en precio usando IQR
Q1 = df['precio'].quantile(0.25)
Q3 = df['precio'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['precio'] < lower_bound) | (df['precio'] > upper_bound)]

print(f"⚠️ OUTLIERS DETECTADOS")
print(f"Total de outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}% del dataset)")
print(f"Rango normal de precios: ${lower_bound:,.0f} - ${upper_bound:,.0f}")
print(f"Precio más alto (outlier): ${outliers['precio'].max():,.0f}")

# Visualizar outliers
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.boxplot(df['precio'])
plt.title('📦 Box Plot - Con Outliers')
plt.ylabel('Precio (USD)')

plt.subplot(1, 2, 2)
df_sin_outliers = df[(df['precio'] >= lower_bound) & (df['precio'] <= upper_bound)]
plt.boxplot(df_sin_outliers['precio'])
plt.title('📦 Box Plot - Sin Outliers')
plt.ylabel('Precio (USD)')

plt.tight_layout()
plt.show()

## 6. Correlaciones

In [ ]:
# Matriz de correlación para variables numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, fmt='.2f', cbar_kws={'shrink': .8})
plt.title('🔗 Matriz de Correlación', fontsize=16)
plt.tight_layout()
plt.show()

# Top correlaciones con precio
precio_corr = correlation_matrix['precio'].abs().sort_values(ascending=False)
print("📊 CORRELACIONES CON PRECIO:")
for var, corr in precio_corr.items():
    if var != 'precio':
        print(f"  {var}: {corr:.3f}")

## 7. Conclusiones

### 📋 Hallazgos Clave:

1. **Distribución de Precios:**
   - El precio promedio es de $XXX,XXX USD
   - La distribución está sesgada hacia la derecha (cola larga)
   - Existen outliers extremos que requieren tratamiento

2. **Variables Más Importantes:**
   - Superficie: Correlación fuerte con el precio
   - Ubicación: Los barrios premium tienen precios 3x más altos
   - Tipo de propiedad: Casas > Departamentos > PH

3. **Calidad de Datos:**
   - Dataset relativamente limpio (< 5% valores faltantes)
   - Outliers identificados (X% del dataset)
   - Variables categóricas necesitan encoding

### 🚀 Próximos Pasos:

- [ ] Limpiar outliers extremos
- [ ] Feature engineering (precio por m², antigüedad, etc.)
- [ ] Encoding de variables categóricas
- [ ] Selección de features para el modelo
- [ ] Split train/test estratificado por barrio

In [ ]:
# Guardar dataset limpio para siguiente notebook
df_clean = df[(df['precio'] >= lower_bound) & (df['precio'] <= upper_bound)]
df_clean.to_csv('../data/processed/propiedades_eda.csv', index=False)

print(f"✅ Dataset procesado guardado")
print(f"Filas originales: {len(df)}")
print(f"Filas después del EDA: {len(df_clean)}")
print(f"Reducción: {len(df)-len(df_clean)} filas ({(len(df)-len(df_clean))/len(df)*100:.1f}%)")